### 加载环境和基础设置

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import time
import logging
import glob
import subprocess
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from datetime import datetime
import shutil

from MacGenerator import *
from RootReader import RootData


# 切换到当前目录，并获取工作目录
os.chdir(sys.path[0])
current_path = os.path.abspath(os.curdir)
# 把r'../../build' 加入到sys.path中
sys.path.append(r'../../build')

# 定义相对路径
rel_dir_root = r'./Data/RawData/'  # 修改为RawData
rel_dir_csv  = r'./Data/CSVData/'
rel_dir_mac  = r'./Data/MacLog/'
rel_dir_log  = r'./Data/RunLog/'
rel_dir_geant4 = r'../build/CompScintSim'

# 获取绝对路径，末尾加上 '/'
abs_dir_root   = os.path.abspath(os.path.join(current_path, rel_dir_root)) + '/'
abs_dir_csv    = os.path.abspath(os.path.join(current_path, rel_dir_csv)) + '/'
abs_dir_mac    = os.path.abspath(os.path.join(current_path, rel_dir_mac)) + '/'
abs_dir_log    = os.path.abspath(os.path.join(current_path, rel_dir_log)) + '/'
abs_dir_geant4 = os.path.abspath(os.path.join(current_path, rel_dir_geant4))

print("RootDir:", abs_dir_root)
print("CSVDir:", abs_dir_csv)
print("MacDir:", abs_dir_mac)
print("LogDir:", abs_dir_log)
print("Geant4Dir:", abs_dir_geant4)

# 确保各个目录存在
for directory in [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log]:
    os.makedirs(directory, exist_ok=True)

# 清除已有日志处理器（避免重复配置）
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# 配置日志记录，日志保存在 abs_dir_log 下
log_filename = "0batch_process.log"
logging.basicConfig(filename=os.path.join(abs_dir_log, log_filename),
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("初始化运行日志")

# 设置全局变量，记录模拟开始前的文件列表
global_before_files = set(os.listdir(abs_dir_root))

class MySim:
    """
    Geant4模拟管理类，支持单粒子和多粒子模拟
    """
    def __init__(self, profile=None, spectrum=None, num_events=None, root_file=None, config=None):
        """
        初始化模拟类
        
        参数：
            profile (str, optional): 模拟配置描述，格式如 'mixed_spectrum_run1'
            spectrum (EnergySpectrum, optional): 能谱对象，如果为None且未指定profile则使用默认能谱
            num_events (int, optional): 模拟事件数，如果为None则使用config['nums']或默认值10
            root_file (str, optional): Root文件保存路径，如果不指定则基于profile自动生成
            config (dict, optional): 配置字典，包含模拟的各种参数
        """
        # 保存配置
        self.config = config or {}
        
        # 确定模拟事件数
        if num_events is not None:
            self.num_events = num_events
        elif 'nums' in self.config:
            self.num_events = self.config['nums']
        else:
            self.num_events = 10
        
        # 根据profile生成文件名，或使用指定的root_file
        if profile:
            self.run_profile = profile
            
            if root_file is None:
                self.root_file = f"{abs_dir_root}{self.run_profile}"
            else:
                self.root_file = root_file
        elif root_file:
            self.root_file = root_file
            self.run_profile = os.path.basename(root_file).replace('.root', '')
        else:
            # 如果没有指定profile和root_file，生成一个随机名称
            timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
            self.run_profile = f"auto_spectrum_{timestamp}"
            self.root_file = f"{abs_dir_root}{self.run_profile}.root"
        
        # 创建能谱对象
        if spectrum is None:
            self.spectrum = EnergySpectrum.create_default_spectrum()
        else:
            self.spectrum = spectrum
        
        # 创建MAC文件生成器
        self.mac_generator = MacFileGenerator(
            spectrum=self.spectrum,
            num_events=self.num_events,
            verbose_level=0,
            root_file=self.root_file
        )
    
    @classmethod
    def from_single_particle(cls, particle, energy, number, suffix="", num_events=None):
        """
        创建单粒子模拟
        
        参数：
            particle (str): 粒子类型，如 'e-', 'proton', 'gamma'
            energy (float): 粒子能量 (MeV)
            number (int): 粒子数量
            suffix (str, optional): 文件名后缀
            num_events (int, optional): 模拟事件数，如果不指定则默认为1
            
        返回：
            MySim: 初始化的模拟对象
        """
        # 验证参数
        valid_particles = ["proton", "e-", "e+", "gamma", "neutron", "alpha"]
        if particle not in valid_particles:
            raise ValueError(f"无效的粒子类型：{particle}")
        
        # 创建能谱
        spectrum = EnergySpectrum()
        spectrum.add_particle(particle, energy, number)
        
        # 创建文件名
        profile = f"{particle}_{energy}MeV_{number}{suffix}"
        
        # 创建模拟对象
        return cls(profile=profile, spectrum=spectrum, num_events=num_events or 1)
    
    @classmethod
    def from_random_spectrum(cls, config=None, profile=None):
        """
        创建随机能谱模拟
        
        参数：
            config (dict, optional): 随机能谱配置
            profile (str, optional): 模拟配置描述，如不指定则自动生成
            
        返回：
            MySim: 初始化的模拟对象
        """
        # 生成随机能谱和获取事件数
        spectrum, num_events = EnergySpectrum.generate_random_spectrum(config)
        
        # 如果未指定profile，自动生成一个
        if profile is None:
            timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
            profile = f"random_spectrum_{timestamp}"
        
        # 创建模拟对象
        return cls(profile=profile, spectrum=spectrum, num_events=num_events, config=config)
    
    def get_profile(self):
        """返回模拟配置描述"""
        return self.run_profile
    
    def write_mac_file(self, dir_mac=abs_dir_mac, mac_name=None):
        """
        生成MAC文件并保存
        
        参数：
            dir_mac (str, optional): MAC文件保存目录
            mac_name (str, optional): MAC文件名，如不指定则使用run_profile
            
        返回：
            str: MAC文件完整路径
        """
        # 如果未指定mac_name，使用run_profile
        if mac_name is None:
            mac_name = f"{self.run_profile}.mac"
        
        dir_mac_file = os.path.join(dir_mac, mac_name)
        
        # 检查目录是否存在
        if not os.path.exists(dir_mac):
            raise FileNotFoundError(f"目录不存在：{dir_mac}")
        
        # 如果文件已存在，自动重命名增加6位时间戳
        if os.path.exists(dir_mac_file):
            print(f"文件已存在：{dir_mac_file}")
            current_time = time.localtime()
            timestamp = time.strftime("%H%M%S", current_time)
            print(f"自动重命名为：{dir_mac_file}_{timestamp}")
            dir_mac_file = f"{dir_mac_file}_{timestamp}"
        
        # 生成MAC文件
        self.mac_generator.generate_mac_file(dir_mac_file)
        
        return dir_mac_file

def run_geant4(dir_geant4, dir_mac_file, log_file, retries=3):
    """
    运行 Geant4 模拟，并记录批处理日志：
      - Geant4 的标准输出和错误输出全部写入日志文件 {abs_dir_log}{log_file}.log
      - 记录模拟是否成功、耗时、生成的新文件等信息，以及使用的 mac 文件名称
      - 如果新生成的 .root 文件名中包含 '_t0' ，说明模拟失败，
        则删除所有 *_t*.root 文件并重新运行模拟（允许重试最多 3 次）
    """
    global global_before_files

    logging.info("##########################################")
    logging.info("开始运行 Geant4 模拟，使用的 mac 文件：%s", dir_mac_file)
    start_time = time.time()
    
    # 设置日志文件路径
    simulation_log_file = os.path.join(abs_dir_log, f"{log_file}.log")
    
    # 运行 Geant4 并将 stdout 和 stderr 重定向到日志文件
    with open(simulation_log_file, "w") as log:
        cmd = [dir_geant4, "-m", dir_mac_file]
        result = subprocess.run(cmd, stdout=log, stderr=log)  # 捕获所有输出到日志文件

        if result.returncode != 0:
            logging.error("Geant4 运行过程中返回非零状态码，可能存在错误，详情见 %s", simulation_log_file)

    elapsed_time = time.time() - start_time
    after_files = set(os.listdir(abs_dir_root))
    new_files = list(after_files - global_before_files)

    logging.info("模拟运行完成，耗时：%.2f 秒", elapsed_time)
    logging.info("生成的新文件：%s", new_files)

    # 检查是否有输出文件包含 '_t0' 且以 .root 结尾（模拟失败的标志）
    failed = any("_t0" in f and f.endswith(".root") for f in new_files)

    if failed:
        logging.error("检测到输出文件中包含 '_t0'，模拟失败。")
        # 删除所有 *_t*.root 文件
        pattern = os.path.join(abs_dir_root, "*_t*.root")
        for file in glob.glob(pattern):
            os.remove(file)
            logging.info("删除失败输出文件：%s", file)
        if retries > 0:
            logging.info("开始重新运行模拟...剩余重试次数：%d", retries)
            return run_geant4(dir_geant4, dir_mac_file, log_file, retries=retries-1)
        else:
            logging.error("重试后模拟仍失败。")
    else:
        logging.info("模拟成功！使用的 mac 文件：%s；耗时：%.2f 秒；生成的新文件：%s", dir_mac_file, elapsed_time, new_files)

    # 更新全局文件列表以便下次对比
    global_before_files = after_files
    
    return new_files




### 测试输出

In [2]:
# 使用示例
def run_examples():
    """
    运行几个示例模拟
    """
    # 设置能量值保留2位小数
    set_energy_decimal_places(2)
    
    # 示例1：单粒子模拟（指定nums=50）
    print("\n示例1：单粒子模拟")
    sim1 = MySim.from_single_particle("e-", 10.0, 100, suffix="_test", num_events=50)
    mac_file1 = sim1.write_mac_file()
    run_geant4(abs_dir_geant4, mac_file1, "single_particle_test")
    
    # 示例2：默认能谱模拟（使用默认nums=10）
    print("\n示例2：默认能谱模拟")
    sim2 = MySim(profile="default_spectrum_test")
    mac_file2 = sim2.write_mac_file()
    run_geant4(abs_dir_geant4, mac_file2, "default_spectrum_test")
    
    # 示例3：随机能谱模拟（指定nums=100）
    print("\n示例3：随机能谱模拟")
    random_config = {
        'e_energy_min': 0.5,
        'e_energy_max': 3.0,
        'e_types_min': 2,
        'e_types_max': 3,
        'p_types_min': 2,
        'p_types_max': 3,
        'g_types_min': 0,  # 不使用伽马粒子
        'g_types_max': 0,
        'use_gamma': False,
        'nums': 100        # 指定事件数为100
    }
    sim3 = MySim.from_random_spectrum(random_config, profile="random_spectrum_test")
    mac_file3 = sim3.write_mac_file()
    run_geant4(abs_dir_geant4, mac_file3, "random_spectrum_test")
    
    # 示例4：自定义能谱模拟（指定nums=200）
    print("\n示例4：自定义能谱模拟")
    custom_spectrum = EnergySpectrum()
    custom_spectrum.add_particle("e-", 1.5, 20)
    custom_spectrum.add_particle("proton", 10.0, 5)
    custom_spectrum.add_particle("gamma", 0.5, 30)
    
    sim4 = MySim(profile="custom_spectrum_test", spectrum=custom_spectrum, num_events=200)
    mac_file4 = sim4.write_mac_file()
    run_geant4(abs_dir_geant4, mac_file4, "custom_spectrum_test")

# run_examples()

### 实际使用

In [3]:
# 清理并重建Data文件夹
for directory in [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log]:
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

#### 自定义能谱

In [ ]:
# 要运行的模拟次数
num_simulations = 3

with tqdm(total=num_simulations, desc="运行模拟", unit="sim") as pbar:
    for i in range(num_simulations):
        # 基础配置
        custom_spectrum = EnergySpectrum()
        custom_spectrum.add_particle("e-", 1.5, 20)
        custom_spectrum.add_particle("proton", 10.0, 5)

        # 生成唯一的模拟标识
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        profile = f"{timestamp}"
        
        try:
            # 创建模拟
            sim = MySim(profile=profile, spectrum=custom_spectrum, num_events=200)
            
            # 写入MAC文件
            mac_file = sim.write_mac_file()
            
            # 运行Geant4模拟
            logging.info(f"开始运行模拟 {i+1}")
            run_geant4(abs_dir_geant4, mac_file, f"batch_sim_{i+1}")
            
            # 直接获取root文件路径
            root_file = os.path.join(abs_dir_root, f"{profile}")
            
            # 检查文件是否存在
            if os.path.exists(root_file+ ".root"):
                logging.info(f"处理root文件: {root_file}")
                
                # 使用RootData处理数据
                root_data = RootData(root_file + ".root")
                
                # 生成CSV文件名并保存
                csv_filename = datetime.now().strftime("%Y%m%d%H%M%S")
                csv_path = root_data.save_simulation_data(abs_dir_csv, csv_filename)
                
                logging.info(f"模拟 {i+1} 完成，CSV文件保存为: {csv_path}")
            else:
                logging.error(f"模拟 {i+1} 的root文件不存在: {root_file}")
        
        except Exception as e:
            logging.error(f"模拟 {i+1} 出错: {str(e)}")
        # 更新进度条
        pbar.update(1)

print("\n批量模拟完成!")


#### 随机能谱

In [ ]:
# 要运行的模拟次数
num_simulations = 3

# 使用tqdm创建进度条
with tqdm(total=num_simulations, desc="运行模拟", unit="sim") as pbar:
    for i in range(num_simulations):
        # 基础配置
        config = {
            'e_types_min': 2,    # 至少2种电子能量
            'e_energy_min': 0.5,  # 电子能量范围0.5-3.0 MeV
            'e_energy_max': 3.0,
            'p_types_min': 2,
            'p_types_max': 3,
            'p_energy_min': 0.5,
            'p_energy_max': 3.0,
            'g_types_min': 0,  
            'g_types_max': 0,
            'nums': 200,
            'use_gamma': False  # 不使用gamma
        }
        
        # 生成唯一的模拟标识
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        profile = f"random_sim_{i+1}_{timestamp}"
        
        try:
            # 创建模拟
            sim = MySim.from_random_spectrum(config, profile=profile)
            
            # 写入MAC文件
            mac_file = sim.write_mac_file()
            
            # 运行Geant4模拟
            logging.info(f"开始运行模拟 {i+1}")
            run_geant4(abs_dir_geant4, mac_file, f"batch_sim_{i+1}")
            
            # 直接获取root文件路径
            root_file = os.path.join(abs_dir_root, f"{profile}")
            
            # 检查文件是否存在
            if os.path.exists(root_file+ ".root"):
                logging.info(f"处理root文件: {root_file}")
                
                # 使用RootData处理数据
                root_data = RootData(root_file + ".root")
                
                # 生成CSV文件名并保存
                csv_filename = datetime.now().strftime("%Y%m%d%H%M%S")
                csv_path = root_data.save_simulation_data(abs_dir_csv, csv_filename)
                
                logging.info(f"模拟 {i+1} 完成，CSV文件保存为: {csv_path}")
            else:
                logging.error(f"模拟 {i+1} 的root文件不存在: {root_file}")
        
        except Exception as e:
            logging.error(f"模拟 {i+1} 出错: {str(e)}")
        # 更新进度条
        pbar.update(1)

print("\n批量模拟完成!")

# 可选：处理当前RawData目录下的所有root文件
# def process_all_root_files():
#     """处理RawData目录下的所有.root文件并保存为CSV"""
#     root_files = [f for f in os.listdir(abs_dir_root) if f.endswith('.root')]
    
#     print(f"\n发现{len(root_files)}个root文件，开始处理...")
    
#     with tqdm(total=len(root_files), desc="处理root文件", unit="file") as pbar:
#         for root_file_name in root_files:
#             try:
#                 root_file_path = os.path.join(abs_dir_root, root_file_name)
#                 root_data = RootData(root_file_path)
                
#                 # 生成CSV文件名（基于root文件名和时间戳）
#                 base_name = os.path.splitext(root_file_name)[0]
#                 csv_filename = f"{base_name}"
                
#                 # 保存CSV文件
#                 csv_path = root_data.save_simulation_data(abs_dir_csv, csv_filename)
#                 logging.info(f"处理完成: {root_file_path} -> {csv_path}")
            
#             except Exception as e:
#                 logging.error(f"处理文件 {root_file_name} 时出错: {str(e)}")
            
#             pbar.update(1)
    
#     print(f"完成处理所有root文件！")


# process_all_root_files()